In [ ]:
from scapy.all import *
filepath = "/home/john/桌面/Anonymous/testflow.pcap"
packages = rdpcap(filepath)[IP]

In [ ]:
datasets = dict()
hashes = set()
for i in range(len(packages)):
    p = packages[i]
    hashid = '' + str(p.proto) + str(p[IP].src) + str(p[IP].dst) + str(p.sport) + str(p.dport)
    hashid = hash(hashid)
    if not hashid in hashes:
        hashes.add(hashid)
        datasets[hashid] = [p]
        continue
    datasets[hashid].append(p)

# print(len(hashes))

biflow = []
showed = set()
for key in datasets.keys():
    temp = []
    showed.add(key)
    temp.append(datasets[key])
    p = datasets[key][0]
    re_key = '' + str(p.proto) + str(p[IP].dst) + str(p[IP].src) + str(p.dport) + str(p.sport)
    re_key = hash(re_key)
    if re_key in showed:
        continue
    if not re_key in hashes:
        biflow.append(temp)
    else:
        temp.append(datasets[re_key])
        biflow.append(temp)
        
print(len(biflow))